# Proyecto 1 Microestructuras y sistemas de Trading

import numpy as np

In [ ]:
import sys, os

In [ ]:
income = (gross_wages +
          taxable_interest +
          (dividends - qualified_dividends) -
          ira_deduction -
          student_loan_interest)